In [1]:
from pyhive import trino
from getpass import getpass
import pandas as pd


from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from datalake_lib.datalake.dllib_spark import DLLibSpark
from datalake_lib.datalake.metadata import Environment

In [2]:
conn = trino.connect(
    host = 'trino.de.in.devneon.com.br',
    port = 443,
    protocol = 'https',
    username = getpass('Insert your User u00xxxx: '),  
    password = getpass('Insert password: '),
)
print(conn)

In [8]:
# Defina as datas de início e fim para a extração
data_inicio = '2023-09-01'
data_fim = '2023-09-10'  # Data atual

# Defina o tamanho do lote (batch size) para extração
batch_size = 100000  # Ajuste conforme necessário

# Calcula o número total de lotes
num_lotes = (pd.to_datetime(data_fim) - pd.to_datetime(data_inicio)).days

# Inicialize uma lista para armazenar os dataframes extraídos
dfs = []

# Inicialize uma variável para rastrear o progresso
progresso = 0

In [10]:
query_loan = f"""
SELECT 
	b.clientid,
	salary,
	incomepercentage,
	firstpaymentdate,
	contractapproveddate,
	referencedate,
	contractstatusdescription,
	financingvalue,
	installmentamount,
	individualinstallmentvalue,
	interestmonthpercentage,
	existsdelayopen,
	maxdelayopen,
	quantityinstallmentspaydelay,
	numberpaidinstallment
FROM neondw_bi.staging_emprestimocontratostatus a
INNER JOIN neondw_bi.dimension_client b on a.clientid = b.clientid
WHERE persontype = 'PF'
AND clientstatus = 'Ativo'
AND contractstatuscode in (8,14)
AND referencedate <= date_add('day', -1, current_date)
"""

In [11]:
df_loan = pd.read_sql(query_loan, conn)

In [1]:
dlspark = DLLibSpark(username, password)
spark = dlspark.spark

NameError: name 'DLLibSpark' is not defined

In [ ]:

# Loop for para dividir a carga em lotes com base nas datas
for _ in range(num_lotes):
    query = f"""
    SELECT 
        b.clientid,
        salary,
        incomepercentage,
        firstpaymentdate,
        contractapproveddate,
        financingvalue,
        installmentamount,
        individualinstallmentvalue,
        interestmonthpercentage,
        existsdelayopen,
        maxdelayopen,
        quantityinstallmentspaydelay,
        numberpaidinstallment
    FROM neondw_bi.staging_emprestimocontratostatus a
    INNER JOIN neondw_bi.dimension_client b on a.clientid = b.clientid
    WHERE persontype = 'PF'
    AND clientstatus = 'Ativo'
    AND contractstatuscode in (8,14)
    AND referencedate >= '{data_inicio}' AND referencedate < '{data_fim}'
    """

    # Execute a consulta e armazene o resultado em um dataframe
    df = pd.read_sql(query, conn)

    # Adicione o dataframe à lista
    dfs.append(df)

    # Atualize as datas para o próximo lote
    data_inicio = data_fim
    data_fim = min(pd.to_datetime(data_fim) + pd.DateOffset(days=1), pd.to_datetime('2023-09-19'))

    # Atualize o progresso
    progresso += 1
    print(f"Progresso: {progresso}/{num_lotes} lotes")

In [7]:
dfs

[]